In [2]:
import os
import yaml

import sys
import pandas as pd
import numpy as np
import pickle
import random
from tqdm import tqdm

In [3]:
path_test = '/home/home/v.moskvoretskii/NLP-DL-Project-hypo-to-hyper/babel_datasets/v2_wnet_test.pickle'
path_pred = '/home/home/v.moskvoretskii/NLP-DL-Project-hypo-to-hyper/babel_datasets/_meta-llama-Llama-2-7b-hfwnet2_filtered_instruct_coma_0'

with open(path_pred, 'rb') as f:
    preds = pickle.load(f)

with open(path_test, 'rb') as f:
    test = pickle.load(f)

len(preds), len(test)

(1274, 1274)

In [4]:
marked = pd.read_csv('../data/assess/merged.csv')

In [5]:
marked

,parent,definition,children,translation,label,label1,label2,label3
0,lowering,the act of causing to become less,tapering,труб,0.0,NaN,0.0,NaN
1,bucket,a roughly cylindrical vessel that is open at t...,"cannikin,dinner pail,kibble,slop pail,wine bucket","канникин, ведро для ужина,киббл, ведро для неч...",1.0,NaN,1.0,NaN
2,touch,a slight but appreciable amount,snuff,snuff,0.0,NaN,0.0,NaN
3,detonator,a mechanical or electrical explosive device or...,blasting cap,дробеструйный колпачок,1.0,NaN,1.0,NaN
4,excess,immoderation as a consequence of going beyond ...,"exorbitance,extravagance","непомерность,экстравагантность",0.0,NaN,0.0,NaN
...,...,...,...,...,...,...,...,...
1269,matter,a vaguely specified concern,least,наименьший,0.0,0.0,NaN,NaN
1270,sausage,-,pork sausage,свиная колбаса,0.0,0.0,NaN,NaN
1271,seasnail,any of several creeping marine gastropods with...,"moon shell,periwinkle,triton,whelk","лунная оболочка,барвинок,тритон,рот",1.0,1.0,NaN,NaN
1272,one-step,an early ballroom dance; precursor to the fox-...,turkey trot,индейка рысь,1.0,1.0,NaN,NaN


In [7]:
marked['id'] = marked['parent'] + marked['children']

In [14]:
marked = marked.drop_duplicates(subset=['id'])

In [26]:
def delete_techniqal(elem, remove=True):
    if isinstance(elem, str):
        if ".n." in elem and remove:
            return elem.split(".")[0].replace("_", " ")
        else:
            return elem.replace("_", " ")

    elif isinstance(elem, list):
        new_words = []
        for word in elem:
            new_words.append(delete_techniqal(word, remove))
        return new_words

easy_test = []
hard_test = []
easy_pred = []
hard_pred = []
hard_ids = []
for i, elem in tqdm(enumerate(test)):
    case = elem["case"]
# processed_term, target = transforms[case](elem)
    
    if isinstance(elem['parents'], list):
        parents = ','.join(delete_techniqal(elem['parents']))
    else:
        parents = delete_techniqal(elem['parents'])
    

    if isinstance(elem['children'], list):
        children = ','.join(delete_techniqal(elem['children']))
    else:
        children = delete_techniqal(elem['children'])

    cur_id = parents + children
    cur_label = int(marked['label'][marked['id'] == cur_id])

    
    if cur_label == 1:
        hard_test.append(elem)
        hard_pred.append(preds[i])
        hard_ids.append(i)
    else:
        easy_test.append(elem)
        easy_pred.append(preds[i])

0it [00:00, ?it/s]/tmp/ipykernel_724393/1480723965.py:35: FutureWarning: Calling int on a single element Series is deprecated and will raise a TypeError in the future. Use int(ser.iloc[0]) instead
  cur_label = int(marked['label'][marked['id'] == cur_id])
1274it [00:00, 4538.35it/s]


In [29]:
[elem for i, elem in enumerate(test) if i in hard_ids]

[{'children': 'covalent bond.n.1',
  'parents': 'chemical bond',
  'grandparents': None,
  'case': 'predict_hypernym',
  'changed': True},
 {'children': 'sight.n.3',
  'parents': 'exteroception',
  'grandparents': None,
  'case': 'predict_hypernym',
  'changed': True},
 {'children': 'cyclopean_masonry.n.1',
  'parents': 'stonework.n.1',
  'grandparents': 'masonry.n.1',
  'case': 'only_child_leaf'},
 {'children': 'western_wood_pewee.n.1',
  'parents': 'pewee.n.1',
  'grandparents': 'New_World_flycatcher.n.1',
  'case': 'only_child_leaf'},
 {'children': ['ichneumon.n.1'],
  'parents': 'mongoose.n.1',
  'grandparents': None,
  'case': 'only_leafs_divided',
  'brothers': ['Indian_mongoose.n.1']},
 {'children': 'common_European_jay.n.1',
  'parents': 'Old_World_jay.n.1',
  'grandparents': 'jay.n.2',
  'case': 'only_child_leaf'},
 {'children': ['abelmosk.n.1',
   'velvetleaf.n.2',
   'chaparral_mallow.n.1',
   'checkerbloom.n.1',
   'tall_mallow.n.1',
   'common_mallow.n.1',
   'false_mallow

In [21]:
len(hard_pred), len(hard_test)

(714, 714)

In [30]:
with open('../../babel_datasets/v2_wnet_test_hard_ids.pickle', 'wb') as f:
    pickle.dump(hard_ids, f)

In [24]:
easy_test[:5]

[{'children': ['backgammon_board.n.1',
   'cribbage_board.n.1',
   'dartboard.n.1',
   'go_board.n.1',
   'monopoly_board.n.1',
   'Ouija.n.1',
   'pegboard.n.1',
   'punchboard.n.1'],
  'parents': 'board.n.9',
  'grandparents': None,
  'case': 'leafs_and_no_leafs'},
 {'children': ['faultlessness.n.1'],
  'parents': 'correctness.n.2',
  'grandparents': None,
  'case': 'only_leafs_divided',
  'brothers': ['political_correctness.n.1']},
 {'children': ['captive.n.3', 'spitfire.n.1'],
  'parents': 'emotional_person.n.1',
  'grandparents': None,
  'case': 'only_leafs_all'},
 {'children': 'semester_hour.n.1',
  'parents': 'credit.n.6',
  'grandparents': 'attainment.n.1',
  'case': 'only_child_leaf'},
 {'children': 'utility.n.1',
  'parents': 'service.n.4',
  'grandparents': 'company.n.1',
  'case': 'simple_triplet_grandparent'}]